# User Intent

The first phase of knowledge graph begins with establishing user intent. You could call this graph ideation. 

## Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [1]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types # For creating message Content/Parts

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [2]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nEnvironment configured.")

ModelResponse(id='chatcmpl-BcZEn9nhDtwbCIJFSEMsO5kPF8Bfb', created=1748531617, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_76544d79cb', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready! How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

Environment configured.


In [3]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message': 'Neo4j is Ready!'}]}


---

## Define the User Intent Agent (`user_intent_agent`)

Input: nothing
Output: `user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.

Configuration:

* `name`: A unique identifier for this agent -- "user_intent_agent_v1"
* `description`: A concise summary of the agent's overall purpose. This becomes crucial later when other agents need to decide whether to delegate tasks to *this* agent.  
* `instruction`: Detailed guidance for the LLM on how to behave, its persona, its goals, and specifically *how and when* to utilize its assigned `tools`.  
* `tools`: A list of tools the agent is allowed to use. In this case, we have `get_user_goal` and `set_user_goal`.



In [4]:
# First, define the instruction to describe what the agent should do
user_intent_agent_instruction = """
        You are an expert at knowledge graph use cases. 
        Your primary goal is to help the user come up with a knowledge graph use case.
        Knowledge graph use cases appear in all industries. Wherever there is data, there's probably a graph.
        
        If the user is unsure where to do, make some suggestions based on classic use cases like:
        - social network involving friends, family, or profressional relationships
        - logistics network with suppliers, customers, and partners
        - recommendation system with customers, products, and purchase patterns
        - fraud detection over multiple accounts with suspicious patterns of transactions
        - pop-culture graphs with movies, books, or music

        A user goal has two components:
        - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
        - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."

        Think carefully and collaborate with the user:
        1. Understand the user's goal, which is a kind_of_graph with description
        2. Ask clarifying questions as needed
        3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record it
        4. Verify with the user that the goal returned by the 'get_perceived_user_goal' tool matches their expectations
        5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
        """


In [11]:
#  Define the tools for the User Intent Agent

def set_perceived_user_goal(kind_of_graph: str, graph_description:str, tool_context: ToolContext):
    """Sets the user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-3 word definition of the kind of graph, for example "recent US patents"
        graph_description: a single paragraph description of the graph, summarizing the user's intent
    """
    user_goal_data = {"kind_of_graph": kind_of_graph, "graph_description": graph_description}
    tool_context.state["perceived_user_goal"] = user_goal_data
    return tool_success("perceived_user_goal", user_goal_data)

def get_perceived_user_goal(tool_context: ToolContext):
    """Returns the user's goal, which is a dictionary containing the kind of graph and its description."""
    if "user_goal" not in tool_context.state:
        return tool_error("user_goal not set. Ask the user to clarify their goal (kind of graph and description).")  
    
    user_goal_data = tool_context.state["user_goal"]

    return tool_success("perceived_user_goal", user_goal_data)



def approve_perceived_user_goal(tool_context: ToolContext):
    """Approves the user's goal, including the kind of graph and its description."""
    if "perceived_user_goal" not in tool_context.state:
        return tool_error("perceived_user_goal not set. Ask the user to clarify their goal (kind of graph and description).")
    
    tool_context.state["approved_user_goal"] = tool_context.state["perceived_user_goal"]
    return tool_success("approved_user_goal", tool_context.state["approved_user_goal"])


user_intent_agent_tools = [get_perceived_user_goal, set_perceived_user_goal, approve_perceived_user_goal]

In [12]:
# Finally, construct the agent

user_intent_agent = Agent(
    name="user_intent_agent_v1",
    model=llm, # defined earlier in a variable
    description="Helps the user ideate on a knowledge graph use case.",
    instruction=user_intent_agent_instruction,
    tools=user_intent_agent_tools,
)

print(f"Agent '{user_intent_agent.name}' created.")

Agent 'user_intent_agent_v1' created.


---

## Interact with the Agent



In [13]:
# Define an Agent Interaction Function

from helpers import make_agent_caller

user_intent_caller = make_agent_caller(user_intent_agent)


In [8]:
# Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await user_intent_caller("What can you do?")


# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: What can you do?
<<< Agent Response: I can help you brainstorm and define a use case for a knowledge graph tailored to your interests or industry needs. Knowledge graphs are versatile and can be applied in various areas such as social networks, logistics, recommendation systems, fraud detection, and pop-culture analyses, among others. We can work together to identify the kind of graph you are interested in and develop a detailed description of its purpose and scope. If you're not sure where to start, I can suggest some classic use cases to consider.


---

Congratulations\! You've successfully built and interacted with your first ADK agent. It understands the user's request, uses a tool to find information, and responds appropriately based on the tool's result.

Next, we'll build a simple multi-agent system.

---
## Finally, An Interactive Conversation

Now, let's make this interactive so you can ask your own questions! Run the cell below. It will prompt you to enter your queries directly.

In [14]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await user_intent_caller(user_query, True)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()


>>> User Query: Let's make a movie graph of movies, actors and acted-in relationships that helps analyze co-acting group dynamics
  [Event] Author: user_intent_agent_v1, Type: Event, Final: True, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='Great idea! A movie graph that analyzes co-acting group dynamics can provide insights into the network of relationships between actors based on their shared movie projects. Let me break this down to understand your goal better:\n\n1. **Kind of Graph**: It seems like you\'re envisioning a "movie actor network".\n\n2. **Description**: The graph would map out movies, the actors in these films, and the relationships between actors who have co-starred, with the aim of analyzing group dynamics and identifying notable actor networks.\n\nDoes this match your intent, or is there anything specific you\'d like to add